In [1]:
import cartopy.crs as ccrs
from datetime import datetime, timedelta,timezone
import gsw
import matplotlib.pyplot as plt
from matplotlib.dates import HourLocator, DateFormatter
import xarray as xr

from ONCToolbox import ONCToolbox
from ONCToolbox.utils import nan_onc_flags, remove_onc_flag_vars
from ONCToolbox.qaqc import location_test,gross_range_test, spike_test, flat_line_test, rate_of_change_test


from ONCToolbox.utils.profilers import split_periods
from ONCToolbox.utils.locations import BCFTerminal

In [2]:
begin_datetime = datetime(2025,7,4)
end_datetime = datetime(2025,7,4,23,59,59,999999)

onct = ONCToolbox()

doi = onct.find_data('TWSB', ['NAV', 'TSG','PVCS'], begin_datetime, end_datetime)

In [3]:
datasets = []
for loc_code, dev_deps in doi.items():
    for dcc, dep_info in dev_deps.items():
        data = onct.get_data(loc_code, dcc, begin_datetime=begin_datetime,end_datetime=end_datetime)
        data = data.drop_vars(['leak_indicator','tank_level'], errors = 'ignore')
        datasets.append(data)
ds = xr.combine_by_coords(datasets, combine_attrs='drop_conflicts')
ds = nan_onc_flags(ds)
ds = remove_onc_flag_vars(ds)

In [4]:
# Linearly Interpolate 
for linterp in ['latitude','longitude','speed_over_ground','outlet_flow','pump_current']:
    ds[linterp] = ds[linterp].interpolate_na(dim = 'time', fill_value = 'extrapolate',max_gap = timedelta(milliseconds = 2000), method = 'linear')

# Nearest Neighbor Interpolate
for ninterp in ['system_state','valve_position']:
    ds[ninterp] = ds[ninterp].interpolate_na(dim = 'time', fill_value = 'extrapolate',max_gap = timedelta(milliseconds = 10000), method = 'nearest')

ds = ds.where(ds.system_state == 1, drop = True)
ds = ds.where(ds.valve_position == 1, drop = True)
ds = ds.where(ds.outlet_flow >= 1, drop = True)
ds = ds.where(ds.pump_current >= 1, drop = True)

ds = ds.drop_vars(['system_state','valve_position', 'heave','pitch', 'roll', 'outlet_flow','pump_current','ship_course','speed_over_ground','true_heading'], errors = 'ignore')

In [5]:
ds = ds.drop_vars(['practical_salinity'], errors = 'ignore')

In [9]:
ds = ds.rename({'conductivity':'sea_water_electrical_conductivity', 'temperature':'sea_water_temperature'})

## QAQC

In [6]:
ds['flag_location'] = location_test(ds.latitude, ds.longitude)


In [8]:
ds

<xarray.Dataset> Size: 8MB
Dimensions:        (time: 195765)
Coordinates:
  * time           (time) datetime64[ms] 2MB 2025-07-04T00:10:59.072000 ... 2...
Data variables:
    conductivity   (time) float64 2MB nan nan 2.969 nan nan ... nan nan nan nan
    temperature    (time) float64 2MB nan nan 19.14 nan nan ... nan nan nan nan
    latitude       (time) float64 2MB 49.0 49.0 49.0 49.0 ... 49.0 49.0 49.0
    longitude      (time) float64 2MB -123.1 -123.1 -123.1 ... -123.1 -123.1
    flag_location  (time) int8 196kB 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1